In [32]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sb
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import mean_squared_error

Data Import

In [13]:
game_data = pd.read_csv("data/clean_data.csv")
game_vo = pd.read_csv("data/Audio_separated.csv")
game_ui = pd.read_csv("data/Lang_separated.csv")

---
Function to seperate languages into individual columns <br>
1/0 as True/False because regression model requires numeric inputs

In [29]:
def seperate(df, column):
    json_table = []
    json_file = df.to_json(orient='records')
    json_file = json.loads(json_file)

    for j in json_file:
        test = {}
        temp = j[column][1:-1]
        test["AppID"] = j["AppID"]
        test["Name"] = j["Name"]
        test["Price"] = j["Price"]
        
        if temp != "":
            temp = temp.split(",")
            for i in temp:
                i = i.replace("'", '')
                test[i] = 1
        
        json_table.append(test)

    res_table = pd.DataFrame(json_table)
    print(res_table.shape)
    print(res_table.columns.tolist())
    res_table.fillna(value=0, inplace=True)

    print(len(json_table))
    return res_table

In [30]:
uis = pd.DataFrame(game_data[["AppID", "Name", "Price", "Supported languages"]])
vos = pd.DataFrame(game_data[["AppID", "Name", "Price", "Full audio languages"]])

sep_game_ui = seperate(uis, "Supported languages")
sep_game_vo = seperate(vos, "Full audio languages")

(16081, 70)
['AppID', 'Name', 'Price', 'English', ' Polish', ' French', ' Italian', ' German', ' Spanish - Spain', ' Portuguese', ' Russian', ' Japanese', ' Korean', ' Turkish', ' Czech', ' Portuguese - Brazil', ' Hungarian', ' Dutch', ' Ukrainian', ' Danish', ' Simplified Chinese', ' Traditional Chinese', ' Arabic', ' Spanish - Latin America', 'Simplified Chinese', ' Thai', ' Finnish', ' Norwegian', ' Romanian', ' Swedish', 'Japanese', ' English', ' Greek', ' Vietnamese', ' Bulgarian', 'Portuguese - Brazil', 'Russian', ' German;', 'Traditional Chinese', 'Polish', 'Korean', 'Danish', 'German', 'Spanish - Spain', '#lang_français', 'Czech', 'French', 'Turkish', 'Dutch', 'Bulgarian', 'Thai', 'Arabic', ' Traditional Chinese (text only)', 'Vietnamese', 'English[b][/b]', ' Italian \\r\\n\\r\\n[b][/b] ', 'Romanian', 'English (full audio)', ' Russian\\r\\n[b][/b]', ' German[b][/b]', ' Spanish - Spain\\r\\n[b][/b]', '  Korean', ' Slovakian', 'English\\r\\nRussian\\r\\nSpanish - Spain\\r\\nJapan

---
Random Forest

In [36]:
# random forest with voice over language
test_regr = RandomForestRegressor(max_features=0.4)

Y = sep_game_vo["Price"]
X = sep_game_vo.drop(columns=["AppID", "Price", "Name"])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)

test_regr.fit(X_train, Y_train)
maybe = test_regr.predict(X_train)
maybe2 = test_regr.predict(X_test)
mse = mean_squared_error(Y_train, maybe)
mse2 = mean_squared_error(Y_test, maybe2)
score = r2_score(Y_train, maybe)
score2 = r2_score(Y_test, maybe2)
print("train score: ",score, " mse: ", mse,  mse**(1/2))
print("test score: ",score2, " mse: ", mse2,  mse2**(1/2))

train score:  0.26204594671093706  mse:  98.23872288613234 9.911544929330258
test score:  0.1294270623374204  mse:  113.60356286785651 10.65849721432888


In [37]:
# random forest with UI language
test_regr = RandomForestRegressor(max_features=0.4)

Y = sep_game_ui["Price"]
X = sep_game_ui.drop(columns=["AppID", "Price", "Name"])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4)

test_regr.fit(X_train, Y_train)
maybe = test_regr.predict(X_train)
maybe2 = test_regr.predict(X_test)
mse = mean_squared_error(Y_train, maybe)
mse2 = mean_squared_error(Y_test, maybe2)
score = r2_score(Y_train, maybe)
score2 = r2_score(Y_test, maybe2)
print("train score: ",score, " mse: ", mse,  mse**(1/2))
print("test score: ",score2, " mse: ", mse2,  mse2**(1/2))

train score:  0.4094917353920644  mse:  77.70585968801275 8.815092721464293
test score:  0.18950533469335606  mse:  107.65535892533212 10.37571004439369
